In [1]:
"""
1. 安装 YOLOX 环境 
"""
%cd /content/
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
# 切换到 0.3.0 版本
!git checkout -b 0.3.0 0.3.0
%pip install -U pip && pip install -r requirements.txt
%pip install -v -e . 

/content
Cloning into 'YOLOX'...
remote: Enumerating objects: 1850, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1850 (delta 17), reused 33 (delta 7), pack-reused 1782
Receiving objects: 100% (1850/1850), 6.90 MiB | 22.07 MiB/s, done.
Resolving deltas: 100% (1082/1082), done.
/content/YOLOX
Switched to a new branch '0.3.0'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
"""
2. 下载 yolox_s 模型
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo
!pip install wget

def download_models():
    """  
    下载 yolox_s 模型
    """
    import os
    import wget
    if not os.path.exists('models/yolox_s.pth'):
        os.mkdir('models')
        wget.download('https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth','models')
    print('models/yolox_s.pth')


download_models()

/content/drive/MyDrive/my-object-detection/yolox-map-demo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=849e01588052248361ba25424e16f11aeac81ab286be811c781d3ff9a1d6be74
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
models/yolox_s.pth


In [3]:
"""
3. 数据集
---
数据集: https://drive.google.com/file/d/12s_QoE9FxqNEgO6c6hgRQ_ySNonQWBza/view?usp=share_link
"""
#------------------
# 我的 Google Drive 路径，本地测试可直接下载数据集，解压到 datasets 文件夹
#------------------
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo

def datasets_download():
    """
    下载数据集 
    """
    import os 
    if not  os.path.exists('datasets'):
        os.mkdir('datasets')
    # 若使用 colab ，可直接挂载 drive ，进行复制操作；否则自行下载，并进行解压
    if not os.path.exists('datasets/shanghai-map-datasets'):
        os.system('cp /content/drive/MyDrive/my-datasets/shanghai-map-datasets.zip ./datasets/')
        # 解压，可自行进行下载，解压到 datasets 文件夹
        os.system('unzip -d datasets/map-datasets datasets/shanghai-map-datasets.zip')
        # 删除，多余文件
        os.system('rm datasets/shanghai-map-datasets.zip')

    
# 下载
datasets_download()

/content/drive/MyDrive/my-object-detection/yolox-map-demo


In [4]:
"""
3.1 数据集整理
---
labelimg  文件整理，将 XML 标注文件和图片放入同一个文件夹中，如 datasets/map-datasets
"""

%mv datasets/map-datasets/annodata/*.xml datasets/map-datasets/
%mv datasets/map-datasets/annodata/*.png datasets/map-datasets/
%rm -r datasets/map-datasets/annodata

In [12]:
"""
3.2 数据集转换 
---
将 labelimg 标注数据集，转换为 coco 数据集，使用 tools/labelimg2coco.py 
"""

%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo


#--------------------------------------------------------
# 注意 classes = ['0','1','2'], 训练图片后缀为 .png
#--------------------------------------------------------

def datasets_init_coco():
    import os 
    if not  os.path.exists('datasets/map'):
        os.mkdir('datasets/map')
    from tools.labelimg2coco import labelimg2coco
    labelimg2coco(
        labelimg_dir='datasets/map-datasets',
        coco_dir='datasets/map',
        classes=['0','1','2'],
        image_suffix='.png'
    )

datasets_init_coco()

/content/drive/MyDrive/my-object-detection/yolox-map-demo
datasets/map/annotations/instances_train2017.json
datasets/map/annotations/instances_val2017.json
datasets/map/annotations/instances_test2017.json
datasets/map/train.txt
datasets/map/val.txt
datasets/map/test.txt
datasets/map/class.txt
train size 21 , val size 6 , test size 3


In [16]:
"""
4. 训练 
---
将 YOLOX/exps/example/custom/yolox_s.py 文件，移动到 coco_exp.py
注意：Google Colab 记得清理下 YOLOX_outputs 下的相关文件
"""

%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo

# %rm -r YOLOX_outputs/crack_exp_yolox_s

!python train.py -f coco_exp.py -d 1 -b 16 --fp16 -o -c models/yolox_s.pth


/content/drive/MyDrive/my-object-detection/yolox-map-demo
2023-02-22 01:18:21 | INFO     | yolox.core.trainer:130 - args: Namespace(batch_size=16, cache=None, ckpt='models/yolox_s.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='coco_exp.py', experiment_name='coco_exp', fp16=True, logger='tensorboard', machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2023-02-22 01:18:21 | INFO     | yolox.core.trainer:131 - exp value:
╒═══════════════════╤════════════════════════════╕
│ keys              │ values                     │
╞═══════════════════╪════════════════════════════╡
│ seed              │ None                       │
├───────────────────┼────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'          │
├───────────────────┼────────────────────────────┤
│ print_interval    │ 10                         │
├───────────────────┼────────────────────────────┤
│ eval_interval     │ 1                          │
├─────

In [18]:
"""
5. 模型评估
"""

%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo
%cp YOLOX_outputs/coco_exp/best_ckpt.pth models/coco_best_ckpt.pth

# 评估命令
!python eval.py -f coco_exp.py -c models/coco_best_ckpt.pth -b 16 -d 1 --conf 0.001 --fp16 --fuse

/content/drive/MyDrive/my-object-detection/yolox-map-demo
2023-02-22 01:26:56 | INFO     | __main__:139 - Args: Namespace(batch_size=16, ckpt='models/coco_best_ckpt.pth', conf=0.001, devices=1, dist_backend='nccl', dist_url=None, exp_file='coco_exp.py', experiment_name='coco_exp', fp16=True, fuse=True, legacy=False, machine_rank=0, name=None, nms=None, num_machines=1, opts=[], seed=None, speed=False, test=False, trt=False, tsize=None)
2023-02-22 01:26:57 | INFO     | __main__:149 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-22 01:26:57 | INFO     | __main__:150 - Model Structure:
YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(


In [20]:
"""
6. 图片测试 
---
修改 coco_demo.py 的 COCO_CLASS 内容
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo

# 测试
!python coco_demo.py image -f coco_exp.py -c models/coco_best_ckpt.pth --path assets/test.png --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

#--------------------------------
# 保存结果
#--------------------------------

# %cd /content/drive/MyDrive/my-object-detection/yolox-map-demo
# %cp YOLOX_outputs/coco_exp/vis_res/2023_02_22_01_31_02/test.png assets/test_result.png


/content/drive/MyDrive/my-object-detection/yolox-map-demo
2023-02-22 01:30:59.047 | INFO     | __main__:main:261 - Args: Namespace(camid=0, ckpt='models/coco_best_ckpt.pth', conf=0.25, demo='image', device='gpu', exp_file='coco_exp.py', experiment_name='coco_exp', fp16=False, fuse=False, legacy=False, name=None, nms=0.45, path='assets/test.png', save_result=True, trt=False, tsize=640)
2023-02-22 01:30:59.362 | INFO     | __main__:main:271 - Model Summary: Params: 8.94M, Gflops: 26.76
2023-02-22 01:31:02.100 | INFO     | __main__:main:284 - loading checkpoint
2023-02-22 01:31:02.373 | INFO     | __main__:main:288 - loaded checkpoint done.
2023-02-22 01:31:03.879 | INFO     | __main__:inference:167 - Infer time: 1.4556s
2023-02-22 01:31:03.907 | INFO     | __main__:image_demo:204 - Saving detection result in ./YOLOX_outputs/coco_exp/vis_res/2023_02_22_01_31_02/test.png


In [22]:
"""
8. 删除缓存
---
删除缓存，减少 Google  Drive 占用
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo
# 删除世代模型结果
%rm -r YOLOX_outputs


/content/drive/MyDrive/my-object-detection/yolox-map-demo


In [23]:
"""
9. 提交代码到 Github
"""
%cd /content/drive/MyDrive/my-object-detection/yolox-map-demo
!git config --global user.name "LABELNET"
!git config --global user.email 1406046087@qq.com 
!git add  .
!git commit -m 'add coco train code'
!git push origin main

/content/drive/MyDrive/my-object-detection/yolox-map-demo
[main 7bce55b] add coco train code
 9 files changed, 330 insertions(+), 5 deletions(-)
 create mode 100644 assets/test_result.png
 create mode 100644 coco-map.ipynb
 create mode 100644 coco_demo.py
 rename yolox_coco_s.py => coco_exp.py (77%)
 delete mode 100644 map-coco.ipynb
 rename map-voc.ipynb => voc-map.ipynb (100%)
 rename demo.py => voc_demo.py (100%)
 rename yolox_voc_s.py => voc_exp.py (100%)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 2 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 398.08 KiB | 7.66 MiB/s, done.
Total 9 (delta 0), reused 0 (delta 0)
To https://github.com/LABELNET/yolox-map-demo.git
   c8724e9..7bce55b  main -> main
